In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read Data

In [116]:
# 為了處理方便，把 'train.csv' 和 'test.csv' 合併起來，'test.csv'的 Weather 欄位用 0 補起來。
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')
test_dataset['Weather'] = np.zeros((len(test_dataset),))

# 以 train_end_idx 作為 'train.csv' 和 'test.csv' 分界列，
train_end_idx = len(train_dataset)
df = pd.concat([train_dataset, test_dataset], sort=False)

In [117]:
normalization = False
one_hot = False
train_model = ['DecisionTree', 'KNeighbors', 'Regression', 'AdaBoost', 'NaiveBayes'] # 'DecisionTree', 'KNeighbors', 'SVM', 'RandomForest', 'Regression', 'AdaBoost', 'NaiveBayes', 'MLPClassifier', 'NN' 

# Preprocessing

In [118]:
X_train.columns

Index(['Loc', 'TempLow', 'TempHigh', 'Steaming', 'Sun', 'WindDir', 'WindSpeed',
       'DayWindDir', 'NightWindDir', 'DayWindSpeed', 'NightWindSpeed',
       'DayHumidity', 'NightHumidity', 'NightPressure', 'DayCloud',
       'NightCloud', 'DayTemp', 'NightTemp', 'RISK_MM'],
      dtype='object')

In [119]:
# features that we don't want
drop_list = ['Date', 'DayPressure', 'DayPressure'] # drop_list = ['Weather', 'Date', 'Loc', 'WindDir', 'DayWindDir', 'NightWindDir']
X_train, y_train = df.drop(columns=['Weather']).drop(columns=drop_list), df['Weather']

In [120]:
columns = X_train.columns
string_list = [col for col in columns if X_train[col].dtype == 'object']

categorical_list = string_list.copy()
categorical_list.append('DayCloud')
categorical_list.append('NightCloud')

numeric_list = [col for col in columns if col not in categorical_list]

In [121]:
# fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15,15))
# plt.suptitle("Correlations", fontsize=20)

# count = 0
# col = 0
# for i in range(4):
#     for j in range(4):
#         axes[i,j].scatter(X_train[y_train==1][numeric_list[col]],X_train[y_train==1][numeric_list[count]],color=["crimson"],s=50,alpha=0.8,label='1')
#         axes[i,j].scatter(X_train[y_train==0][numeric_list[col]],X_train[y_train==0][numeric_list[count]],color=["chartreuse"],s=20,label="0")
#         axes[i,j].set_xlabel(numeric_list[col])
#         axes[i,j].set_ylabel(numeric_list[count])
#         axes[i,j].legend(prop={'size': 10},loc="lower left") #圖例setting
#         col += 1

In [122]:
# Handle missing values
# df = df.fillna(0)
X_train['WindDir'] = X_train['WindDir'].fillna(X_train['DayWindDir'])
X_train['DayWindDir'] = X_train['DayWindDir'].fillna(X_train['WindDir'])
X_train['NightWindDir'] = X_train['NightWindDir'].fillna(X_train['WindDir'])

X_train['WindSpeed'] = X_train['WindSpeed'].fillna(X_train['DayWindSpeed'])
X_train['DayWindSpeed'] = X_train['DayWindSpeed'].fillna(X_train['WindSpeed'])
X_train['NightWindSpeed'] = X_train['NightWindSpeed'].fillna(X_train['WindSpeed'])

X_train['TempLow'] = X_train['TempLow'].fillna(X_train['DayTemp'])
X_train['TempHigh'] = X_train['TempHigh'].fillna(X_train['NightTemp'])
X_train['DayTemp'] = X_train['DayTemp'].fillna(X_train['TempHigh'])
X_train['NightTemp'] = X_train['NightTemp'].fillna(X_train['TempHigh'])

# if 'DayPressure' in drop_list or 'NightPressure' in drop_list:
#   X_train['DayPressure'] = X_train['DayPressure'].fillna(X_train['NightPressure'])
#   X_train['NightPressure'] = X_train['NightPressure'].fillna(X_train['NightPressure'])

for d in string_list:
    X_train[d] = X_train[d].fillna('Unknown')

In [123]:
from sklearn.preprocessing import LabelEncoder

for d in string_list:
    labelEncoder = LabelEncoder()
    labelEncoder.fit(X_train[d])
    X_train[d] = labelEncoder.transform(X_train[d]).astype(float)


In [124]:
mean = X_train.mean()
X_train = X_train.fillna(mean)

In [125]:
if one_hot:
  X_train = pd.get_dummies(data=X_train, columns=categorical_list)
# X_train

In [126]:
X_train

,Loc,TempLow,TempHigh,Steaming,Sun,WindDir,WindSpeed,DayWindDir,NightWindDir,DayWindSpeed,NightWindSpeed,DayHumidity,NightHumidity,NightPressure,DayCloud,NightCloud,DayTemp,NightTemp,RISK_MM
0,42.0,19.5,19.600000,5.694960,0.500000,1.0,61.0,1.0,1.0,31.0,22.0,86.641375,41.810006,1021.90000,4.154681,7.00000,19.6,19.600000,1.772145
1,9.0,2.1,15.700000,5.701668,5.700000,0.0,26.0,13.0,9.0,9.0,7.0,53.275715,76.598048,1020.80000,4.000000,1.00000,9.0,14.100000,0.000000
2,49.0,20.9,36.100000,5.694960,8.003251,8.0,39.0,8.0,12.0,31.0,17.0,15.471574,52.163254,1014.90000,4.167266,4.27812,24.2,33.900000,0.000000
3,40.0,11.5,23.900000,5.694960,7.993227,5.0,35.0,14.0,13.0,7.0,17.0,46.442398,67.664733,1015.47264,4.167266,4.27812,13.8,21.800000,0.200000
4,15.0,8.2,20.500000,3.200000,8.003251,3.0,69.0,3.0,15.0,69.0,69.0,51.830889,51.770240,1001.20000,7.000000,7.00000,20.5,17.600000,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34839,42.0,9.6,21.500000,8.000000,12.000000,11.0,31.0,2.0,9.0,17.0,9.0,39.878487,57.490952,1022.80000,0.000000,6.00000,14.3,19.000000,0.000000
34840,9.0,11.6,31.400000,5.694960,7.993227,5.0,39.0,13.0,9.0,7.0,39.0,51.902205,70.743790,1016.30000,4.167266,4.27812,19.3,31.400000,0.000000
34841,21.0,22.2,34.500000,5.701668,7.600000,5.0,39.0,5.0,3.0,26.0,39.0,30.743840,64.412890,1008.50000,7.000000,7.00000,25.2,34.500000,0.000000
34842,20.0,16.4,43.500000,7.000000,12.700000,1.0,43.0,13.0,13.0,19.0,19.0,28.084916,63.565166,1010.50000,0.000000,4.27812,28.9,43.500000,1.772145


# Split Data

In [127]:
from sklearn.model_selection import train_test_split

X_col = X_train.columns

# Split data
X_test = X_train.values[train_end_idx:, :]
X_train, X_val, y_train, y_val = train_test_split(X_train.values[:train_end_idx], y_train.values[:train_end_idx], test_size=0.2)

X_train = pd.DataFrame(data=X_train, columns=X_col)
X_val = pd.DataFrame(data=X_val, columns=X_col)
X_test = pd.DataFrame(data=X_test, columns=X_col)

# Over-Sampling

In [128]:
# print(y_train.value_counts())
print(np.unique(y_train, return_counts=True))

# Over sampling
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE(random_state=42).fit_resample(X_train, y_train)

# Under sampling
from imblearn.under_sampling import TomekLinks
X_train, y_train = TomekLinks().fit_resample(X_train, y_train)

# print(y_train.value_counts())
print(np.unique(y_train, return_counts=True))

(array([0., 1.]), array([25126,  2998]))
(array([0., 1.]), array([25126, 25116]))


# Normalization

In [129]:
from sklearn.preprocessing import StandardScaler

# Normalization
if normalization:
  features = X_train[numeric_list]
  scaler = StandardScaler().fit(features.values)
  features = scaler.transform(features.values)
  X_train[numeric_list] = features
  # X_train

  val_features = X_val[numeric_list]
  val_features = scaler.transform(val_features.values)
  X_val[numeric_list] = val_features
  # X_val

  test_features = X_test[numeric_list]
  test_features = scaler.transform(test_features.values)
  X_test[numeric_list] = test_features
  # X_test

# Dimension Reduction

In [130]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=130)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_val = pca.transform(X_val)
# X_test = pca.transform(X_test)

In [131]:
X_train.shape

(50242, 19)

# Train

In [132]:
from sklearn.metrics import accuracy_score, f1_score

In [133]:
from sklearn.tree import DecisionTreeClassifier

#train tree model
if 'DecisionTree' in train_model:
  model = DecisionTreeClassifier()
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('DecisionTree.csv',  index_label = 'Id')

Accuracy: 0.825228
f1-score: 0.255603


In [134]:
from sklearn.neighbors import KNeighborsClassifier

#train k-neighbors model
if 'KNeighbors' in train_model:
  model = KNeighborsClassifier()
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('KNeighbors.csv',  index_label = 'Id')

Accuracy: 0.712173
f1-score: 0.299169


In [135]:
from sklearn.svm import SVC

# train SVM
if 'SVM' in train_model:
  model = SVC(kernel='rbf', C=2)
  model.fit(X_train, y_train)

  # predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('SVM.csv',  index_label = 'Id')

In [136]:
from sklearn.linear_model import LogisticRegression

#train Regression
if 'Regression' in train_model:
  model = LogisticRegression(max_iter=1000)
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('Regression.csv',  index_label = 'Id')

Accuracy: 0.722127
f1-score: 0.347797


In [137]:
from sklearn.ensemble import RandomForestClassifier

#train random forest
if 'RandomForest' in train_model:
  model = RandomForestClassifier(criterion='entropy', n_estimators=200, max_depth=10)
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('RandomForest.csv',  index_label = 'Id')

In [138]:
from sklearn.ensemble import AdaBoostClassifier

#train AdaBoost
if 'AdaBoost' in train_model:
  model = AdaBoostClassifier(n_estimators=100, learning_rate=0.7)
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('AdaBoost.csv',  index_label = 'Id')

Accuracy: 0.833049
f1-score: 0.369495


In [139]:
from sklearn.naive_bayes import GaussianNB

#train naive-bayes model
if 'NaiveBayes' in train_model:
  model = GaussianNB()
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('NaiveBayes.csv',  index_label = 'Id')

Accuracy: 0.722838
f1-score: 0.317816


In [140]:
from sklearn.neural_network import MLPClassifier

#train MLP classifier
if 'MLPClassifier' in train_model:
  model = MLPClassifier(hidden_layer_sizes=(32,16,5), activation='relu', solver='adam', batch_size=200, learning_rate_init=1e-3, max_iter=200)
  model.fit(X_train, y_train)

  #predict
  y_pred = model.predict(X_val)
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  ans_pred = model.predict(X_test)
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('MLPClassifier.csv',  index_label = 'Id')

In [141]:
import tensorflow as tf
from tensorflow.keras import layers, models

if 'NN' in train_model:
  model = models.Sequential()
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(8, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=[tf.keras.metrics.BinaryAccuracy(),])

  history = model.fit(X_train, y_train, batch_size=200, epochs=20, validation_data=(X_val, y_val))

  #predict
  y_pred = model.predict(X_val)
  y_pred = (y_pred > 0.5).astype('int32').flatten()
  print('Accuracy: %f' % accuracy_score(y_val, y_pred))
  print('f1-score: %f' % f1_score(y_val, y_pred))

  # plot lost curve
  plt.plot(history.history['loss'], label='loss')
  # plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')

  ans_pred = model.predict(X_test)
  ans_pred = (ans_pred > 0.5).astype('int32').flatten()
  df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
  df_sap.to_csv('NN.csv',  index_label = 'Id')